# MBTI별 포스팅 모으기

In [8]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [9]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

#input_file = r'.\data\training'
#allFile_list = glob.glob(os.path.join(input_file, 'mbti_*'))
#for file in allFile_list:
#    csv = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
#    cleanMbti = csv['posts'].str.replace('[^A-Za-z\s]+', '')

# OverSampling

In [10]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

from imblearn.over_sampling import RandomOverSampler

In [11]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

print(types)

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
y = y_data.values.reshape(-1, 1)

X_resampled, y_resampled = RandomOverSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)


      posts
type       
ENFJ    190
ENFP    675
ENTJ    231
ENTP    685
ESFJ     42
ESFP     48
ESTJ     39
ESTP     89
INFJ   1470
INFP   1832
INTJ   1091
INTP   1304
ISFJ    166
ISFP    271
ISTJ    205
ISTP    337


Counter({'INFJ': 1832,
         'ENTP': 1832,
         'INTP': 1832,
         'INTJ': 1832,
         'ENTJ': 1832,
         'ENFJ': 1832,
         'INFP': 1832,
         'ENFP': 1832,
         'ISFP': 1832,
         'ISTP': 1832,
         'ISFJ': 1832,
         'ISTJ': 1832,
         'ESTP': 1832,
         'ESFP': 1832,
         'ESTJ': 1832,
         'ESFJ': 1832})

# tokenize and stemming

In [12]:
mbti= dataCombine
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [13]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]

#vectorization (tfidf)
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())


stemming 시작
stemming 완료
[[0.16126    0.         0.         ... 0.         0.         0.        ]
 [0.15308429 0.         0.         ... 0.         0.         0.        ]
 [0.0701916  0.         0.         ... 0.         0.         0.        ]
 ...
 [0.14387306 0.         0.         ... 0.         0.         0.        ]
 [0.25156428 0.         0.         ... 0.         0.         0.        ]
 [0.20342093 0.         0.         ... 0.         0.         0.        ]]


In [15]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model